In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint


In [9]:
train_dir = "train"  
test_dir = "test" 

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    color_mode="grayscale",
    batch_size=32,
    class_mode="sparse",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    color_mode="grayscale",
    batch_size=32,
    class_mode="sparse",
    subset="validation"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    color_mode="grayscale",
    batch_size=32,
    class_mode="sparse"
)

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(128, (3, 3), activation="relu"),
        Conv2D(256, (3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(256, activation="relu"),
        Dropout(0.5),
        Dense(7, activation="softmax")
    ])
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

In [5]:
save_path = r"G:/Codes/ShadowFox/ShadowFOX/AIML/Begineers/FacialExpression/model_weights.h5"

model = build_model()

if os.path.exists(save_path):
    try:
        model.load_weights(save_path)
        print("Model weights loaded successfully!")
    except Exception as e:
        print(f"Error loading model weights: {e}")
        print("Starting training from scratch.")
else:
    print(f"No weights found at {save_path}. Starting training from scratch.")

Model weights loaded successfully!


In [12]:
checkpoint = ModelCheckpoint(save_path, monitor="val_loss", save_best_only=True, mode="min")

history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[checkpoint]
)

Epoch 1/50
718/718 [==============================] - 64s 87ms/step - loss: 1.7950 - accuracy: 0.2530 - val_loss: 1.7870 - val_accuracy: 0.2513
Epoch 2/50
718/718 [==============================] - 63s 88ms/step - loss: 1.7847 - accuracy: 0.2567 - val_loss: 1.7911 - val_accuracy: 0.2517
Epoch 3/50
718/718 [==============================] - 63s 88ms/step - loss: 1.7618 - accuracy: 0.2697 - val_loss: 1.7616 - val_accuracy: 0.2548
Epoch 4/50
718/718 [==============================] - 63s 87ms/step - loss: 1.7285 - accuracy: 0.2884 - val_loss: 1.6939 - val_accuracy: 0.3177
Epoch 5/50
718/718 [==============================] - 62s 87ms/step - loss: 1.6967 - accuracy: 0.3149 - val_loss: 1.6337 - val_accuracy: 0.3416
Epoch 6/50
718/718 [==============================] - 62s 87ms/step - loss: 1.6599 - accuracy: 0.3296 - val_loss: 1.5858 - val_accuracy: 0.3689
Epoch 7/50
718/718 [==============================] - 62s 86ms/step - loss: 1.6225 - accuracy: 0.3550 - val_loss: 1.5605 - val_accuracy:

In [13]:
model.save_weights(save_path)
print(f"Model weights saved to {save_path}.")

Model weights saved to G:/Codes/ShadowFox/ShadowFOX/AIML/Begineers/FacialExpression/model_weights.h5.


In [ ]:
def predict_expression(face, model):
    face_resized = cv2.resize(face, (64, 64))
    face_normalized = face_resized / 255.0
    face_expanded = np.expand_dims(face_normalized, axis=0)  # Add batch dimension
    face_expanded = np.expand_dims(face_expanded, axis=-1)  # Add channel dimension (grayscale)
    prediction = model.predict(face_expanded)
    return np.argmax(prediction)

# Label map for expressions (adjust as per your dataset)
label_map = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Open the camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Camera not found!")
    exit()

print("Camera is successfully opened!")

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

while True:
    ret, frame = cap.read()

    if not ret:
        print("Failed to grab frame.")
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the face region
        face = gray[y:y + h, x:x + w]

        # Predict the expression
        expression_idx = predict_expression(face, model)
        label = label_map[expression_idx]

        # Draw rectangle and label around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the frame
    cv2.imshow("Facial Expression Detection", frame)

    # Break the loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

Camera is successfully opened!
1/1 [==============================] - 0s 21ms/step
